To save in google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


**Packages**

In [2]:
! pip install datasets --quiet
! pip install evaluate --quiet
! pip install rouge_score --quiet
! pip install sacrebleu --quiet
! pip install transformers --quiet
! pip install -q sentencepiece --quiet
! pip install summarizer --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.5 MB/s eta 

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

from datasets import load_dataset
import evaluate

from pprint import pprint

**Data**

In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/W266FinalProject/Datasets/xl_sum_sample_train.csv')
val_df = pd.read_csv('/content/drive/MyDrive/W266FinalProject/Datasets/xl_sum_sample_val.csv')
test_df = pd.read_csv('/content/drive/MyDrive/W266FinalProject/Datasets/xl_sum_sample_test.csv')

In [5]:
print(f'train shape: {train_df.shape}')
print(f'val size: {val_df.shape}')
print(f'test size: {test_df.shape}')

train shape: (1000, 2)
val size: (100, 2)
test size: (100, 2)


In [6]:
train_df.head(n=2)

,text,summary
0,By Rebecca Ricks & Johnny O'SheaBBC Spotlight ...,"During the spring, at the height of the Covid-..."
1,"By Rachel SchraerBBC Reality Check So, why did...","The parents of five-year-old Tafida Raqeeb, wh..."


**T5 Model**

##### 1.) Load and set up model

In [7]:
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

t5model = TFT5ForConditionalGeneration.from_pretrained("t5-base")
t5tokenizer = T5Tokenizer.from_pretrained("t5-base")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [8]:
## Set up model params

summary_max_length = 512
target_max_length = 128

##### 3.) Test model

In [9]:
rouge = evaluate.load('rouge')

In [10]:
chrf = evaluate.load("chrf")

In [11]:
test_df.shape

(100, 2)

In [12]:
r1 = []
r2 = []
rL = []
rLs = []
chrfs = []

for i in test_df.index:

    T5ARTICLE_TO_SUMMARIZE = test_df['text'][i]

    inputs = t5tokenizer(T5ARTICLE_TO_SUMMARIZE, 
                         max_length=summary_max_length, 
                         truncation=True, 
                         return_tensors="tf")

    summary_ids = t5model.generate(inputs["input_ids"], max_length=target_max_length)

    candidate = t5tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

    ref = [test_df['summary'][i]]

    rouge_results = rouge.compute(predictions=candidate,
                                  references=ref)

    r1.append(rouge_results['rouge1'])
    r2.append(rouge_results['rouge2'])
    rL.append(rouge_results['rougeL'])
    rLs.append(rouge_results['rougeLsum'])
    
    chrf_results = chrf.compute(predictions=candidate,
                                references=ref)
    chrfs.append(chrf_results['score'])

    if i in np.arange(0, 50, 100):
        print(i)

0


In [13]:
print('rouge1 average :', np.mean(r1))
print('rouge2 average :', np.mean(r2))
print('rougeL average :', np.mean(rL))
print('rougeLs average :', np.mean(rLs))
print('chrf average :', np.mean(chrfs))

rouge1 average : 0.19809182836136244
rouge2 average : 0.03253749902763041
rougeL average : 0.13468246215931587
rougeLs average : 0.13468246215931587
chrf average : 26.984854771913675


In [14]:
data = {'rouge1': r1, 'rouge2': r2, 'rogueL': rL, 'rogueLs': rLs, 'chrf': chrfs}

scores = pd.DataFrame(data)

scores.to_csv(r'/content/drive/MyDrive/W266FinalProject/model_results/no_finetuned_T5_scores_samples_maxlength512.csv', index=False)